# Optic Flow
Inspired by: A Robust Road Vanishing Point Detection Adapted to the
Real-World Driving Scenes

Based on the:
1. analysis,
2. stable motion detection
3. stationary point-based motion vector selection
4. angle-based RANSAC
(RANdom SAmple Consensus) voting

In [74]:
import cv2
import numpy as np
import os

# create temp folder
if not os.path.exists('temp'):
    os.makedirs('temp')

In [75]:
file = "test_20s_video.MP4"
cap = cv2.VideoCapture(file)
total_number_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [76]:
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # mask bottom
    gray[-400:] = 0
    # blur
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    return gray

def get_frame(cap, frame_number):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    processed_frame = preprocess_frame(frame)
    return frame, processed_frame

# Video Builder

In [77]:
class VideoBuilder:
    def __init__(self, filename, fps):
        self.fps = fps
        self.output_file = filename
        self.recorder = None

    def add_frame(self, frame):
        if self.recorder is None:
            self.recorder = cv2.VideoWriter(self.output_file, cv2.VideoWriter_fourcc(*'XVID'), self.fps, (frame.shape[1], frame.shape[0]))

        self.recorder.write(frame)

    def stop_recording(self):
        if self.recorder is not None:
            self.recorder.release()
            self.recorder = None


# Motion Vector Detection

In [78]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 500,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

### Corner Feature Detection
Detect corners using Shi-Tomasi corner detection algorithm.

In [79]:
# !! any changes to this function will require re-running the entire script and clearing temp cache
all_corners = []

corner_cache = {}

def get_points_of_frame(frame_number):
    if frame_number in corner_cache:
        return corner_cache.get(frame_number)
    
    original_frame, frame = get_frame(cap, frame_number)

    # shi-tomasi corner detection
    corners = cv2.goodFeaturesToTrack(frame, mask = None, **feature_params)

    if corners is not None:
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(original_frame, (int(x), int(y)), 3, 255, -1)

    corner_cache[frame_number] = corners
    return corners

### Feature Tracking using Lucas-Kanade Optical Flow

In [80]:
# recorder
record = VideoBuilder(fps=cap.get(cv2.CAP_PROP_FPS), filename='all_optic_flow.avi')

# initial frame
p0 = get_points_of_frame(0)
_, last_frame = get_frame(cap, 0)


# random colours to label different lines
color = np.random.randint(0, 255, (400000, 3))


# Dictionary to store trajectories of each point
trajectories = {tuple(p.ravel()): [p] for p in p0}

for i in range(1, total_number_of_frames):
    frame, frame_gray = get_frame(cap, i)
    
    mask = np.zeros_like(frame)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(last_frame, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    new_trajectories = {}
    for new, old in zip(good_new, good_old):
        start_position = tuple(old.ravel())  # Use the original position as the key
        next_iterations_start_position = tuple(new.ravel())  # Use the new position as the key for the next iteration
        if start_position in trajectories:
            # Append new position to the trajectory with the same start position
            trajectories[start_position].append(new)
            # Keep this trajectory in the new set of trajectories
            new_trajectories[next_iterations_start_position] = trajectories[start_position]

    # Replace old trajectories with updated ones that exclude lost points
    trajectories = new_trajectories

    # Draw the full trajectories
    for start_position, points in trajectories.items():
        color_idx = hash(start_position) % len(color)  # Get a consistent color for each start position
        for j in range(1, len(points)):
            a, b = points[j].ravel()
            c, d = points[j - 1].ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[color_idx].tolist(), 2)

    # Draw current points as circles
    for start_position, points in trajectories.items():
        color_idx = hash(start_position) % len(color)  # Re-use the consistent color
        a, b = points[-1].ravel()  # Last known position
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[color_idx].tolist(), -1)

    # draw on image
    frame = cv2.add(frame, mask)

    # save image
    record.add_frame(frame)

    last_frame = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    if len(p0) < 400:
        new_points = get_points_of_frame(i)
        p0 = np.concatenate((p0, new_points), axis=0)
        for new_p in new_points:
            start_position = tuple(new_p.ravel())
            if start_position not in trajectories:
                trajectories[start_position] = [new_p]

record.stop_recording()